In [5]:
import numpy as np
import pandas as pd 
import jieba 
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from collections import Counter
from math import isnan
from sklearn.feature_extraction.text import TfidfTransformer
txt = pd.read_excel('数据比较好的.xlsx')
Y = txt['标签']
jieba.load_userdict(Y)
stopwords = [line.strip() for line in open('../input/stopword.txt', 'r', encoding='utf-8').readlines()] 
list_des = txt['商品描述'].tolist()
X = []
for i in range(len(list_des)):
    des = list_des[i].replace(' ','')
    seg_list = jieba.lcut(des)
    word_list = [] 
    for seg in seg_list:
        if seg not in stopwords:
            word_list.append(seg)
    X.append((' ').join(word_list))
txt['商品描述'] = pd.Series(X)  

In [6]:
for i in range(len(txt)):
    length = len(txt.loc[i]['商品描述'].split())
    if length >15:
        txt.drop(index = i,inplace=True)

In [7]:
frame = pd.DataFrame(columns=['商品描述','标签'])
dic = Counter(Y)
for i in dic.keys():
    data = txt.loc[txt['标签'] == i]
    if dic[i] > 200:
        frame = pd.concat([frame,data[:200]],axis = 0)
        continue
    elif dic[i]>=30:
        frame = pd.concat([frame,txt.loc[txt['标签'] == i]],axis = 0)
frame = frame.reset_index(drop=True)
c_num = len(Counter(frame['标签']).keys())
print('数据量：%d'%frame.shape[0],'分类数目：%d'%len(Counter(frame['标签']).keys()))

数据量：40391 分类数目：511


In [4]:
#frame.to_csv('20-200.csv',encoding='utf_8_sig')

In [8]:
frematrix = CountVectorizer().fit_transform(frame['商品描述'])
tfidf = TfidfVectorizer().fit_transform(frame['商品描述'])
lr = LogisticRegression()
X_train,X_test,Y_train,Y_test = train_test_split(frematrix,frame['标签'],test_size = 0.1)
lr.fit(X_train,Y_train)
y_pre = lr.predict(X_test)
print(np.mean(y_pre == Y_test))

lr = LogisticRegression()
X_train,X_test,Y_train,Y_test = train_test_split(tfidf,frame['标签'],test_size = 0.1)
lr.fit(X_train,Y_train)
y_pre = lr.predict(X_test)
print(np.mean(y_pre == Y_test))

0.8344059405940594
0.8175742574257425


In [9]:
word_txt = []
for label in Counter(frame['标签']).keys():
    same = frame.loc[frame['标签'] == label]
    word = []
    for i in same['商品描述']:
        word += i.split()
    word_txt.append(word)
from gensim.models import word2vec
#sentences = word2vec.Text8Corpus('分词拿去做word2vect.txt')
model = word2vec.Word2Vec(word_txt,size = 200,window = 900,min_count = 3)

c:\python3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [10]:
chi2_words = [line.strip('/n').split() for line in open('bag_words.txt','r').readlines()]

In [11]:
chi2_words_2 = chi2_words[0] + chi2_words[1]

In [12]:
word2vec_matrix = []
for line in range(frame.shape[0]):
    vector = []
    num = 0
    words = frame['商品描述'][line].split()
    for word in words:
        try:
            temp = model[word]
        except:
            continue 
        else:
            vector += list(temp)
            num += 1
    common_words = list(set(words)&set(chi2_words_2))
    #print(line,common_words)
    state = 0
    if len(common_words) != 0:
        while num <15:
            for c_word in common_words:
                try:
                    temp = model[c_word]
                except:
                    continue 
                else:
                    state = 1
                    vector += list(temp)
                    num += 1
                    if num == 15:
                        break
            if state == 0:
                vector = vector + [0]*200*(15 - num)
                break
    else:
        vector = vector + [0]*200*(15 - num)
    word2vec_matrix.append(vector)

c:\python3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
c:\python3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [13]:
from sklearn import preprocessing 
le = preprocessing.LabelEncoder()
y_label = le.fit_transform(frame['标签'])

In [14]:
import tensorflow as tf
from tensorflow.contrib import rnn
import random

input_size = 200
timestep_size = 15 
hidden_size = 256
layer_num = 2
class_num = c_num
learning_rate = 0.001
with tf.name_scope('inputs'):
    keep_prob = tf.placeholder(tf.float32)
    batch_size = tf.placeholder(tf.int32, [], name='batch_size_input')
    _X = tf.placeholder(tf.float32,[None,3000])
    y = tf.placeholder(tf.int64,[None])
    x = tf.reshape(_X,[-1,timestep_size,input_size])

with tf.name_scope("fw"):
    stacked_rnn_fw = []
    for _ in range(layer_num):
        fw_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size, forget_bias=1.0, state_is_tuple=True)
        stacked_rnn_fw.append(fw_cell)
    lstm_fw_cell_m = tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.MultiRNNCell(stacked_rnn_fw), output_keep_prob= keep_prob)
    #lstm_fw_cell_m = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_rnn_fw, state_is_tuple=True)

with tf.name_scope("bw"):
    stacked_rnn_bw = []
    for _ in range(layer_num):
        bw_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size, forget_bias=1.0, state_is_tuple=True)
        stacked_rnn_bw.append(bw_cell)
    lstm_bw_cell_m = tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.MultiRNNCell(stacked_rnn_bw), output_keep_prob= keep_prob)
    #lstm_bw_cell_m = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_rnn_bw, state_is_tuple=True)
    
with tf.variable_scope("output",reuse = tf.AUTO_REUSE):
    x = tf.unstack(x, 15, axis=1)
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell_m, lstm_bw_cell_m, x, dtype=tf.float32)

In [15]:
attention_size = 256
with tf.name_scope('attention'), tf.variable_scope('attention'):
    attention_w = tf.Variable(tf.truncated_normal([2*hidden_size, attention_size], stddev=0.1), name='attention_w')
    attention_b = tf.Variable(tf.constant(0.1, shape=[attention_size]), name='attention_b')
    u_list = []
    for t in range(15):
        u_t = tf.tanh(tf.matmul(outputs[t], attention_w) + attention_b) 
        u_list.append(u_t)
    u_w = tf.Variable(tf.truncated_normal([attention_size, 1], stddev=0.1), name='attention_uw')
    attn_z = []
    for t in range(15):
        z_t = tf.matmul(u_list[t], u_w)
        attn_z.append(z_t)
    # transform to batch_size * sequence_length
    attn_zconcat = tf.concat(attn_z, axis=1)
    alpha = tf.nn.softmax(attn_zconcat)
    # transform to sequence_length * batch_size * 1 , same rank as outputs
    alpha_trans = tf.reshape(tf.transpose(alpha, [1,0]), [15, -1, 1])
    final_output = tf.reduce_sum(outputs * alpha_trans, 0)
with tf.variable_scope("result",reuse = tf.AUTO_REUSE):
    fc_w = tf.Variable(tf.truncated_normal([2*hidden_size, class_num], stddev=0.1), name='fc_w')
    fc_b = tf.Variable(tf.zeros([class_num]), name='fc_b')
    #self.final_output = outputs[-1]
    # 用于分类任务, outputs取最终一个时刻的输出
    logits = tf.matmul(final_output, fc_w) + fc_b
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits+1e-10,labels = y)
    original_cost_function = tf.reduce_mean(loss)
    
    tv = tf.trainable_variables()
    regularization_cost = 0.001* tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv ])
    cost = original_cost_function + regularization_cost

    train_op = tf.train.AdamOptimizer(learning_rate).minimize(original_cost_function)
with tf.name_scope("accuracy"):
    prediction = tf.argmax(logits,1)
    correct_prediction = tf.equal(prediction,y)
    correct_num=tf.reduce_sum(tf.cast(correct_prediction,tf.float32))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32),name="accuracy")

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(word2vec_matrix,y_label,test_size = 0.1)
y_pre_list = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    listnum = []
    Iter = 0
    for i in range(5001):
        _batch_size = 1000
        if len(listnum)<_batch_size:
            listnum = list(range(len(X_train)))
            Iter += 1
        temp_x =[]
        temp_y = []
        for _ in range(_batch_size):
            j = random.randint(0,len(listnum)-1) #生成一个包括0，len(listnum)-1之间的随机数
            temp_x.append(X_train[listnum[j]])
            temp_y.append(Y_train[listnum[j]])
            del listnum[j]
        # temp_x 是一个1000*1200维度的向量
        if (i)%200 == 0:
            y_pre_list.append(sess.run(prediction, feed_dict={_X:X_test, y: Y_test, keep_prob: 1.0, batch_size:len(X_test)}))
            train_accuracy = sess.run(accuracy, feed_dict={_X:X_test, y: Y_test, keep_prob: 1.0, batch_size:len(X_test)})
            print("Iter %d: step %d, training accuracy %g" % ( Iter,(i+1), train_accuracy))
        sess.run(train_op,feed_dict={_X:temp_x,y:temp_y,keep_prob: 0.3, batch_size: _batch_size})

Iter 1: step 1, training accuracy 0.000990099
Iter 6: step 201, training accuracy 0.811634
Iter 12: step 401, training accuracy 0.834653
Iter 17: step 601, training accuracy 0.834901
Iter 23: step 801, training accuracy 0.836386
Iter 28: step 1001, training accuracy 0.834406
Iter 34: step 1201, training accuracy 0.834158
Iter 39: step 1401, training accuracy 0.838614
Iter 45: step 1601, training accuracy 0.833911


In [28]:
y_pre = le.inverse_transform(y_pre_list[-1])
y_true = le.inverse_transform(Y_test)

c:\python3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\python3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [31]:
for i in range(len(y_pre)):
    if y_pre[i] != y_true[i]:
        print(y_pre[i],y_true[i])

西装 礼服
鸭舌帽 遮阳帽
连裤袜 表演服
置物架 挂件
背心 文胸
纹身贴纸 纹身贴
展示架 展示盒
单鞋 帆布鞋
长裤 哈伦裤
休闲裤 长裤
娃娃衫 背心
短裤 打底裤
吊坠 手链
短裤 打底裤
收纳包 手提包
手链 手环
铅笔 用品
围巾 咖啡
泳衣 游泳衣
证件套 挂绳
礼服 高跟鞋
发饰 头饰
打底衫 背心
零钱包 卡包
模型 公仔
手表带 表带
球衣 足球服
贴纸 指甲贴纸
项链 手绳
雪纺衫 小衫
手提包 帆布包
拍照道具 道具
开衫 半身裙
连体衣 短裤
钱包 卡包
毛绒玩具 玩偶
凉拖鞋 单鞋
汤碗 餐具
文件夹 写字板
手串 圆珠
T恤衫 短袖
上衣 抹胸
电源 移动电源
手提袋 帆布袋
中长裙 旗袍
耳饰 耳钉
开关 传感器
皮套 橡皮筋
桌椅 家具
鞋子 鞋套
手办 玩具车
音箱 扩音器
背景纸 道具
交换机 教材
摆件 烟灰缸
卫衣 夹克
绘本 教材
蓝牙耳机 耳机
相框 钥匙链
乐福鞋 单鞋
沙发 沙发床
耳夹 布贴
打底裙 短裙
半身裙 短裙
女包 手提包
支架 充电器
铝膜气球 气球
挂坠 手链
泳衣 泳装
手链 橡皮筋
椅子 凳子
对讲机 电钻
棉袜 船袜
抹胸 吊带背心
文胸 抹胸
椅子 座椅
针织开衫 毛衣
充电器 适配器
衣服 连体衣
表演服 公主裙
耳饰 耳夹
长裤 阔腿裤
水杯 道具
连衣裙 旗袍
贴纸 标签纸
清洗机 喷头
裙子 连衣裙
女袜 船袜
鞋子 帆布鞋
分装瓶 喷雾瓶
长裙 半裙
支架 手机支架
防尘罩 收纳袋
椅子 桌子
贴膜 手机膜
储物柜 柜子
储物箱 收纳箱
凉鞋 凉拖
单肩包 手机包
短裤 泳裤
餐盘 盘子
防晒服 防晒衣
手柄 充电器
休闲鞋 单鞋
女包 手提包
遮阳帽 鸭舌帽
灯箱 道具
收纳盒 整理架
头饰 礼物
针织衫 T恤衫
数据线 连接线
牛仔短裤 短裤
雪纺衫 打底衫
内衣 睡衣
礼服 晚礼服
钱包 充值卡
首饰盒 收纳篮
衬衣 衬衫
装饰画 壁画
话筒 音响
模型 人偶
垫子 脚垫
鞋垫 袜子
保护套 手机壳
高跟鞋 鞋垫
束口袋 口袋
行车记录仪 记录仪
皮鞋 女单鞋
高跟鞋 女单鞋
手环 腕带
保护套 手机壳
七分裤 休闲裤
睡裙 内衣
头饰 工具
连衣裙 包臀裙
T恤 短袖
手柄 底座
储物盒 收纳箱
裤子 牛仔裤
拉链 外套
零钱包 收纳包
头饰 仿真花
雨伞 晴雨伞
休闲裤 